In [1]:
import pandas as pd

df = pd.read_csv("malicious_phish.csv")

df['malicious'] = df['type'].apply(lambda x: 1 if x!= 'benign' else 0)


In [2]:
df.head(5)

,url,type,malicious
0,br-icloud.com.br,phishing,1
1,mp3raid.com/music/krizz_kaliko.html,benign,0
2,bopsecrets.org/rexroth/cr/1.htm,benign,0
3,http://www.garage-pirenne.be/index.php?option=...,defacement,1
4,http://adventure-nicaragua.net/index.php?optio...,defacement,1


In [3]:
datos_insertar = df[df['malicious'] == 1]

In [4]:
datos_revisar = df[df['malicious'] == 0]

In [5]:
datos_insertar.to_csv("insercionBF.csv", index=False)
datos_revisar.to_csv("revisarBF.csv", index=False)

In [7]:
datos_revisar.size

1284309

In [8]:
datos_insertar.size

669264

In [ ]:
## Iterar sobre los elementos de un dataframe, para poder insertarlos al bloom

for elem in datos_insertar['url']:
    print(elem)

In [ ]:
## iterar sobre los elementos del dataframe, para revisar si están o no
# sabemos por fact que ninguno de estos están en el bloom

for elem in datos_revisar['url']:
    print(elem)